<h1><span style="color:red">Simple Variable Calculations</span></h1>

### This sample notebook reads numeric variables from a survey dataset and lets users compute a new numeric variable, add it to a new survey version, and publish the survey to the user's surveys gallery

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf myclone
!git clone --depth 1 "https://github.com/SteveLamont/Suave_on_Colab.git" myclone

Cloning into 'myclone'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 87 (delta 24), reused 59 (delta 14), pack-reused 0
Unpacking objects: 100% (87/87), 564.74 KiB | 7.74 MiB/s, done.


In [ ]:
%cd /content/myclone/helpers
!git pull

/content/myclone/helpers
git: 'pul' is not a git command. See 'git --help'.

The most similar commands are
	pull
	push


In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import operator

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

def printmd(string):
    display(Markdown(string))

absolutePath = "/content/drive/MyDrive/suave/"

# local imports
# import sys
# sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint


## 1. Read the survey file and extract numeric variables

In [ ]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' in n}
printmd("<b><span style='color:red'>Numeric variables:</span></b>")

for key, value in var_list.items():
    print(key, ' : ', value)


## 2. Define a new variable using the form ##

In [ ]:
printmd("<b><span style='color:red'>Select the number of operations you would like to perform, then run the next cell</span></b>")
printmd("<b><span style='color:red'>For instance, 3+5*6 is 2 operations</span></b>")
num_ops = widgets.BoundedIntText(value=1,min=1,max=100,step=1,description='Number of Operations:',disabled=False, style={'description_width': 'max-content'})
display(num_ops)

In [ ]:
# Define a new Variable
var_dict = {}

ops = {"+": operator.add, "-": operator.sub, "/": operator.truediv, "*": operator.mul} 

printmd("<b><span style='color:red'>Select variables and the operation, then run the next cell</span></b>")
keep_track = 0
for i in range(num_ops.value):
    if i == 0:
        var_dict[str(0)] = widgets.Dropdown(options=var_list.keys())
        var_dict[str(1)] = widgets.Dropdown(options=['/','*','+','-'])
        var_dict[str(2)] = widgets.Dropdown(options=var_list.keys())
        display(var_dict[str(0)])
        display(var_dict[str(1)])
        display(var_dict[str(2)])
        keep_track +=3
    else:
        var_dict[str(keep_track)] = widgets.Dropdown(options=['/','*','+','-'])
        var_dict[str(keep_track+1)] = widgets.Dropdown(options=var_list.keys())
        display(var_dict[str(keep_track)])
        display(var_dict[str(keep_track+1)])
        keep_track += 2

## 3. Edit variable name as needed

In [ ]:
# Give a Name to the New Variable
def f(Var_Name):
    return Var_Name
keep_track = 0
new_name = ''
for i in range(num_ops.value):
    if i == 0:
        new_name = new_name + var_dict[str(0)].value + var_dict[str(1)].value + var_dict[str(2)].value
        keep_track +=3
    else:
        new_name = new_name + var_dict[str(keep_track)].value + var_dict[str(keep_track+1)].value
        keep_track += 2
    
newvar = interact(f, Var_Name=new_name +'#number');

printmd("<b><span style='color:red'>After defining variable name hit Enter, then run the next cell</span></b>")


## 4. Compute the new variable and format it for SuAVE

In [ ]:
try:
    temp = 0
    keep_track = 0
    
    for i in range(num_ops.value):
        if i == 0:
            temp = ops[str(var_dict[str(1)].value)](df[var_list[str(var_dict[str(0)].value)]],df[var_list[str(var_dict[str(2)].value)]])
            keep_track +=3
        else:
            temp = ops[str(var_dict[str(1)].value)](temp,df[var_list[str(var_dict[str(keep_track+1)].value)]])
            keep_track += 2
    df[newvar.widget.result] = temp
        
# make sure there are no illegal NaN type values in this #number variable
    df[newvar.widget.result].fillna('',inplace=True)
    df.replace(['None', 'nan', np.inf, -np.inf], np.nan, inplace=True)
    df[newvar.widget.result] = pd.to_numeric(df[newvar.widget.result], errors='coerce', downcast='float')
    df[newvar.widget.result] = df[newvar.widget.result].apply(lambda x: '{:.6f}'.format(x))
    df.replace('nan', "", inplace=True)
    printmd("<b><span style='color:red'>New variable computed, and appended to the data frame as the last variable.</span></b>")

except:
    printmd("<b><span style='color:red'>!! Cannot compute. One or both variables contain non-numeric values!!</span></b>")
    printmd("<b>" + var_list[a.value] +"</b>")
    print(df[var_list[a.value]].describe())
    printmd("<b>" + var_list[c.value] +"</b>")
    print(df[var_list[c.value]].describe())


In [ ]:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df.columns):
        display(df.drop(['geometry'],axis=1))
    else:
        display(df)
    

## 5. Save the new version of CSV file, and give a name to new survey

In [ ]:
new_file = suaveint.save_csv_file(df, absolutePath, csv_file)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)


## 6. Generate the survey and create survey URL

In [ ]:
suaveint.create_survey( survey_url, new_file, survey_name, dzc_file, user, csv_file, view, views )